In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import csv

### Which event codes matter here:
*  made field goal attempts
*  made final free throw attempt
*  missed final free throw attempt that results in a defensive rebound
*  missed field goal attempt that results in a defensive rebound
* (5) turnover
* (13) end of time period.
* (8) substitutions

## Notes
* Deadball rebounds are always offensive, so they do not matter.

Keep in mind we have to keep track of each players:
* Number of offensive possesions
* Number of defensive possessions
* Team points allowed while on court
* Team points scored while on court

## Assumptions

* In the Game_Lineup.txt file, the 0th period lists all players for each team, and then each period lists the starters for that period.

#### Clean the Object data

In [ ]:
game_lineup = pd.read_csv('Game_Lineup.txt',sep='\s+')
play_by_play = pd.read_csv('Play_by_Play.txt',sep='\s+')
event_codes = pd.read_csv('Event_Codes.txt',sep='\s+',)

In [ ]:
for col in game_lineup.columns.values:
    if game_lineup[col].dtype=='object':
        game_lineup[col] = game_lineup[col].str.strip()
        
for col in play_by_play.columns.values:
    if play_by_play[col].dtype=='object':
        play_by_play[col] = play_by_play[col].str.strip()
        
for col in event_codes.columns.values:
    if event_codes[col].dtype=='object':
        event_codes[col] = event_codes[col].str.strip()

#### Order the games by Event_Num and create NaN values to clear up data.

In [ ]:
play_by_play = play_by_play.sort_values(["Game_id","Period","PC_Time","Event_Num"],ascending=[True,True,False,True])

play_by_play = play_by_play.replace('0370a0d090da0d0edc6319f120187e0e','NBA')
play_by_play = play_by_play.replace('1473d70e5646a26de3c52aa1abd85b1f','NBA')

### Map each player to the correct team.

In [ ]:
team_mapper = {}
for player in play_by_play['Person1'].unique():
    if len(game_lineup[game_lineup['Person_id']==player]["Team_id"].unique()) > 0:
        team_mapper[player] = game_lineup[game_lineup['Person_id']==player]["Team_id"].unique()[0]

# correct team values, merge with original Team_id identifier
team_id2 = play_by_play.Person1.apply(lambda x: team_mapper[x] if x in team_mapper else np.NaN) 
play_by_play.Team_id = team_id2.fillna(play_by_play.Team_id)  # use the mapped team identifier where possible, if not possible go with what the data gave us

### Lets try to get this working for one single game.

In [ ]:
game_id = "9b718cc561cecf4da6d0632def573409"
game1 = play_by_play[play_by_play.Game_id==game_id].reset_index()

cols = game1.columns.tolist()

A possession is ended by 
* (1) made field goal attempts
* (2) made final free throw attempt
* (3) missed final free throw attempt that results in a defensive rebound,
* (4) missed field goal attempt that results in a defensive rebound
* (5) turnover
* (6) end of time period.

In [ ]:
players = pd.DataFrame(index=game_lineup[(game_lineup.Game_id==game_id) & (game_lineup.Period==0)]['Person_id'].tolist(),columns=['PossessionsFor','PointsFor','PossessionsAgainst','PointsAgainst'])
players['Person_id'] = players.index
players = players.fillna(0)
players['Team_id'] = game_lineup[(game_lineup.Game_id==game_id) & (game_lineup.Period==0)]['Team_id'].tolist()

incoming_sub = False
entering = []
leaving = []

for row in game1.iterrows():

    print(row[0])
    instance = row[1].tolist()
    event_msg_type = instance[cols.index("Event_Msg_Type")]
    
    if incoming_sub and instance[cols.index("PC_Time")] != substitution_time:
        
        [on_court.append(player) for player in entering]
        [on_court.remove(player) for player in leaving]
        incoming_sub = False # reset the indicator
        entering = []
        leaving = []

    
    if event_msg_type==1: # made shot
         
        points = instance[cols.index("Option1")]
        team_off = instance[cols.index("Team_id")]

        players.PointsFor = players.PointsFor + (players.where((players.Person_id.isin(on_court)) & (players.Team_id==team_off))['PointsFor'].apply(lambda x: x*0) + points).fillna(0)
        players.PointsAgainst = players.PointsAgainst + (players.where((players.Person_id.isin(on_court)) & (players.Team_id!=team_off))['PointsAgainst'].apply(lambda x: x*0) + points).fillna(0)
        
        players.PossessionsFor = players.PossessionsFor + (players.where((players.Person_id.isin(on_court)) & (players.Team_id==team_off))['PossessionsFor'].apply(lambda x: x*0) + 1).fillna(0)
        players.PossessionsAgainst = players.PossessionsAgainst + (players.where((players.Person_id.isin(on_court)) & (players.Team_id!=team_off))['PossessionsAgainst'].apply(lambda x: x*0) + 1).fillna(0)
        
    elif event_msg_type==2 and game1.iloc[(row[0]+1)]["Event_Msg_Type"]==4: # missed shot followed by a rebound
        
        team_off = instance[cols.index("Team_id")]
        team_reb = game1.iloc[(row[0]+1)]["Team_id"] # team that got the rebound
        
        if team_off != team_reb: # defensive rebound, so a change of possession occured
            
            players.PossessionsFor = players.PossessionsFor + (players.where((players.Person_id.isin(on_court)) & (players.Team_id==team_off))['PossessionsFor'].apply(lambda x: x*0) + 1).fillna(0)
            players.PossessionsAgainst = players.PossessionsAgainst + (players.where((players.Person_id.isin(on_court)) & (players.Team_id!=team_off))['PossessionsAgainst'].apply(lambda x: x*0) + 1).fillna(0)
        
    elif event_msg_type==3 and instance[cols.index("Action_Type")] in [10,12,15,19,20,22,26,29]: # all of these action types signify the last free throw
        
        # last free throw taken, option1 has the 
        # total points accumulated on the taken free throws
        
        team_off = instance[cols.index("Team_id")]
        
        points = instance[cols.index("Option1")]
        players.PointsFor = players.PointsFor + (players.where((players.Person_id.isin(on_court)) & (players.Team_id==team_off))['PointsFor'].apply(lambda x: x*0) + points).fillna(0)
        players.PointsAgainst = players.PointsAgainst + (players.where((players.Person_id.isin(on_court)) & (players.Team_id!=team_off))['PointsAgainst'].apply(lambda x: x*0) + points).fillna(0)
        
        if game1.iloc[(row[0]+1)]["Event_Msg_Type"]==4 and game1.iloc[(row[0]+1)]["Team_id"]==team_off:  # this means that the last free throw attempt was missed, and a offensive rebound occured
            
            None # do nothing
            
        else: # otherwise, either a defensive rebound occured or the shot was made, ending the possession
            
            players.PossessionsFor = players.PossessionsFor + (players.where((players.Person_id.isin(on_court)) & (players.Team_id==team_off))['PossessionsFor'].apply(lambda x: x*0) + 1).fillna(0)
            players.PossessionsAgainst = players.PossessionsAgainst + (players.where((players.Person_id.isin(on_court)) & (players.Team_id!=team_off))['PossessionsAgainst'].apply(lambda x: x*0) + 1).fillna(0)
        
    elif event_msg_type==5: # the team associated with the event number is the team who LOST possesion
        
        team_off = instance[cols.index("Team_id")]
        players.PossessionsFor = players.PossessionsFor + (players.where((players.Person_id.isin(on_court)) & (players.Team_id==team_off))['PossessionsFor'].apply(lambda x: x*0) + 1).fillna(0)
        players.PossessionsAgainst = players.PossessionsAgainst + (players.where((players.Person_id.isin(on_court)) & (players.Team_id!=team_off))['PossessionsAgainst'].apply(lambda x: x*0) + 1).fillna(0)
        
    elif event_msg_type==8: # substitution! Person1 LEAVES the court, Person2 ENTERS
        
        # new players are not technically on the court until the game clock starts moving
        
        incoming_sub = True
        leave = instance[cols.index("Person1")]
        enter = instance[cols.index("Person2")]
        substitution_time = instance[cols.index("PC_Time")]
        
        entering.append(enter)
        leaving.append(leave)
            
    
    elif event_msg_type==12: # start of period

        period = instance[cols.index("Period")]
        on_court = game_lineup[(game_lineup.Game_id==game_id) & (game_lineup.Period==period)]['Person_id'].tolist()
        
    elif event_msg_type==13: # end of period
        
        players.PossessionsFor = players.PossessionsFor + (players.where((players.Person_id.isin(on_court)) & (players.Team_id==team_off))['PossessionsFor'].apply(lambda x: x*0) + 1).fillna(0)
        players.PossessionsAgainst = players.PossessionsAgainst + (players.where((players.Person_id.isin(on_court)) & (players.Team_id!=team_off))['PossessionsAgainst'].apply(lambda x: x*0) + 1).fillna(0)

In [ ]:
players['Offensive_Rating'] = 100* players.PointsFor/players.PossessionsFor
players['Defensive_Rating'] = 100* players.PointsAgainst/players.PossessionsAgainst

In [ ]:
# players = players.reset_index()

with open('spencer_tibbitts.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)

lines = []
for i in range(len(players)):
    lines.append([game_id]+[players.iloc[i]["Team_id"]] + [players.iloc[i]["Person_id"]] + [players.iloc[i]["Offensive_Efficiency"]] + [players.iloc[i]["Defensive_Efficiency"]])

with open('spencer_tibbitts.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(["Game_id","Team_id","Person_id","Offensive_Efficiency","Defensive_Efficiency"])
    writer.writerows(lines)